# 1. Setup

In [1]:
%load_ext autoreload

In [3]:
%autoreload 2
import json
import logging
import os
from datetime import datetime
from pathlib import Path

import pandas as pd
import regex as re
import typer
from dateutils import relativedelta
from unidecode import unidecode

from analysis.config import *
from analysis.preprocessing import map_progress, tokenizer_lemma
from analysis.utils import *
from src.custom_logger import get_logger

# Removendo aviso de debug
os.environ["PYDEVD_WARN_SLOW_RESOLVE_TIMEOUT"] = "3000"
# Removendo limitação de print de dfs
# pd.set_option("display.max_rows", 500)
# pd.set_option("display.max_columns", 500)
# pd.set_option("display.width", 1000)

# Caminho da pasta contendo os csvs
data_path = "data/2023/01/19/"
places_file = "data/places.csv"
input_file = "input/hotels_23_01_19.csv"
reports_folder = Path("./reports")
Path(reports_folder).mkdir(exist_ok=True)
data_folder = Path("./data/prep")
Path(data_folder).mkdir(exist_ok=True)


In [4]:
from tqdm import tqdm
tqdm.pandas()

In [5]:
from itertools import chain

join_str = lambda x: "\n".join(list(x)) if x is not None else ""
join_list = lambda x: list(chain.from_iterable(x))

# 2. Prep

In [4]:
%timeit
%autoreload 2
from analysis.prep import read_data
from analysis.prep import prep_complete

df_merge = read_data(input_file, data_path)
df_prep = prep_complete(df_merge)
save_df(df_prep, "data/prep", "df_prep")

drop duplicates
likes
trip_type_travel_group
lattitude and longitude
relative dates


100%|██████████| 270097/270097 [00:22<00:00, 12209.46it/s]


Other ratings


100%|██████████| 270097/270097 [00:11<00:00, 22963.98it/s]


Topics


...

Text


100%|██████████| 270097/270097 [00:10<00:00, 26811.12it/s] 


tokens


100%|██████████| 270097/270097 [00:18<00:00, 14925.18it/s]


data/prep/df_prep_2023-06-22_20-52-11_175568.pq


In [5]:
%autoreload 2
from analysis.prep import features
import pandas as pd

df_prep = pd.read_parquet("data/prep/df_prep_2023-06-22_20-52-11_175568.pq")
df_prep = df_prep[features]

In [6]:
df_prep.iloc[0]

feature_id                                                    0x0:0xcb0d68682790adde
name                                                                Atlântico Center
retrieval_date_metadata                                   2023-01-19 21:46:33.484494
state                                                                             RJ
region                                                                       SUDESTE
latitude                                                                  -22.898854
longitude                                                                 -43.179802
stars                                                                              0
overall_rating                                                                   2.9
n_reviews                                                                       1397
topic_names                        [tv, VLT, box, ponto, museu do amanhã, tubo, t...
topic_counts                                    [69, 31, 22, 17, 

# 3. Text Prep

In [7]:
df_prep["has_text"] = ~df_prep.text.isna()
df_prep["has_response_text"] = ~df_prep.response_text.isna()

In [8]:
df_prep.text = df_prep.text.fillna("")
df_prep.response_text = df_prep.response_text.fillna("")

In [9]:
df_prep.text.head()

0    Bom preco ,Ótima Localização,atendimento bom ,...
1    Eu ,nunca vi um quarto de hotel tão sujo na mi...
2    Hotel de baixíssima qualidade no atendimento, ...
3    Péssimo atendimento, muito clássico do Rio de ...
4                                                     
Name: text, dtype: object

In [10]:
def basic_cleanning(text):
    # Removing unnecessary spaces
    text = re.sub(r"\s+", r" ", text)
    # Removing dupplicated punctuation
    text = re.sub(r"[.]+" ,r".",text)
    return text

df_prep["text"] = df_prep["text"].progress_apply(basic_cleanning)
df_prep["response_text"] = df_prep["response_text"].progress_apply(basic_cleanning)

100%|██████████| 270097/270097 [00:05<00:00, 51918.09it/s]


In [11]:
import re
from unidecode import unidecode

text = df_prep.text.iloc[0]

print("text_input", text)

def normalize(text):
    text = unidecode(text).lower()
    return text

print("text_out", normalize(text))

text_input Bom preco ,Ótima Localização,atendimento bom ,quarto espaçoso com ar condicionado . Banheiro com toalhas ,chuveiro quebrado só o cano ,café da manhã com poucas opções. Se arrumar o chuveiro e melhorar o café,dá pra cobrar um pouco mais caro a diária.
text_out bom preco ,otima localizacao,atendimento bom ,quarto espacoso com ar condicionado . banheiro com toalhas ,chuveiro quebrado so o cano ,cafe da manha com poucas opcoes. se arrumar o chuveiro e melhorar o cafe,da pra cobrar um pouco mais caro a diaria.


In [12]:
df_prep["text_normalized"] = df_prep["text"].progress_apply(normalize)
df_prep["response_text_normalized"] = df_prep["response_text"].progress_apply(normalize)

100%|██████████| 270097/270097 [00:07<00:00, 36107.85it/s]


In [13]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import sent_tokenize
# nltk.download("punkt")
# nltk.download("stopwords")
# nltk.download("wordnet")

stopwords = stopwords.words("portuguese")
stm = PorterStemmer()
wnl = WordNetLemmatizer()

def sentence_tokenizer(sentence, reduc="stemmer", min_len=3, stopwords=stopwords):
    words = wordpunct_tokenize(sentence)
    # Removendo pontuação e tornando lowercase
    words = [word.lower() for word in words if word.isalpha()]
    # Removendo stopwords
    words = [word for word in words if word not in stopwords]
    # Selecting and applying reducer
    reduc_f = wnl.lemmatize if reduc == "lemmatizer" else stm.stem
    words = [reduc_f(word) for word in words]
    # Min length cut
    words = [word for word in words if min_len <= len(word)]
    return words

def review_tokenizer(text):
    sentences = text.split(".")
    tokens = [sentence_tokenizer(s) for s in sentences]
    return tokens

text = df_prep.text.iloc[0]
print("text_out", review_tokenizer(text))

text_out [['bom', 'preco', 'ótima', 'localização', 'atendimento', 'bom', 'quarto', 'espaçoso', 'condicionado'], ['banheiro', 'toalha', 'chuveiro', 'quebrado', 'cano', 'café', 'manhã', 'pouca', 'opçõ'], ['arrumar', 'chuveiro', 'melhorar', 'café', 'pra', 'cobrar', 'pouco', 'caro', 'diária'], []]


In [14]:
df_prep.loc[:, "text_tokens"] =              df_prep["text_normalized"].progress_apply(review_tokenizer)
df_prep.loc[:, "text_tokens_len"] =          df_prep["text_tokens"].apply(len)
df_prep.loc[:, "response_text_tokens"] =     df_prep["response_text_normalized"].progress_apply(review_tokenizer)
df_prep.loc[:, "response_text_tokens_len"] = df_prep["response_text_tokens"].apply(len)

100%|██████████| 270097/270097 [00:48<00:00, 5580.62it/s] 


In [15]:
df_prep.highlights.head()


0    [Bom preço]
1           None
2           None
3    [Bom preço]
4           None
Name: highlights, dtype: object

In [16]:
df_prep.highlights = df_prep.highlights.apply(lambda x: ", ".join(x) if x is not None else "")

In [17]:
df_prep["highlights_normalized"] = df_prep["highlights"].progress_apply(normalize)

100%|██████████| 270097/270097 [00:00<00:00, 649229.02it/s]


In [18]:
df_prep.loc[:, "highlights_tokens"] = df_prep["highlights_normalized"].progress_apply(lambda x: [sentence_tokenizer(x)])

100%|██████████| 270097/270097 [00:01<00:00, 188184.52it/s]


In [19]:
df_prep.highlights.head()

0    Bom preço
1             
2             
3    Bom preço
4             
Name: highlights, dtype: object

In [20]:
df_prep.highlights_tokens.head()

0    [[bom, preco]]
1              [[]]
2              [[]]
3    [[bom, preco]]
4              [[]]
Name: highlights_tokens, dtype: object

# 3. Aggregation

In [21]:
df_prep.columns

Index(['feature_id', 'name', 'retrieval_date_metadata', 'state', 'region',
       'latitude', 'longitude', 'stars', 'overall_rating', 'n_reviews',
       'topic_names', 'topic_counts', 'review_id', 'retrieval_date',
       'review_date', 'response_date', 'rating', 'likes', 'rooms', 'locale',
       'service', 'user_id', 'user_name', 'user_is_local_guide',
       'user_reviews', 'user_photos', 'highlights', 'trip_type_travel_group',
       'text', 'response_text', 'text_is_other_language',
       'response_text_is_other_language', 'text_tokens', 'text_tokens_len',
       'response_text_tokens', 'response_text_tokens_len', 'has_text',
       'has_response_text', 'text_normalized', 'response_text_normalized',
       'highlights_normalized', 'highlights_tokens'],
      dtype='object')

In [22]:
from itertools import chain

join_str = lambda x: "\n".join(list(x)) if x is not None else ""
join_list = lambda x: list(chain.from_iterable(x))

df_hotels = df_prep.groupby("name").agg(
    review_count = ("review_id", "count"),
    other_language_count = ("text_is_other_language", "sum"),
    text_count = ("has_text","sum"),
    text_sum = ("text",join_str),
    text_normalized_sum = ("text_normalized",join_str),
    text_tokens_sum = ("text_tokens", join_list),
    response_text_count =          ("has_response_text","sum"),
    response_text_sum =            ("response_text",join_str),
    response_text_normalized_sum = ("response_text_normalized",join_str),
    response_text_tokens_sum =     ("response_text_tokens", join_list),
    highlights_sum = ("highlights", join_str),
    highlights_tokens_sum = ("highlights_tokens", join_list),
).reset_index()

In [23]:
df_hotels = df_hotels.replace(regex=r"\n+",value=r" \n ").replace(regex=r"[.]+",value=r" . ")

In [25]:
df_hotels.shape

(50, 13)

In [26]:
df_hotels.iloc[0]

name                                                                  Acqua Lokos
review_count                                                                 8542
other_language_count                                                          350
text_count                                                                   3994
text_sum                        O parque tá muito ruim,desde a entrada!Piscina...
text_normalized_sum             o parque ta muito ruim,desde a entrada!piscina...
text_tokens_sum                 [[parqu, ruim, desd, entrada, piscina, area, c...
response_text_count                                                           126
response_text_sum                \n Oie Douglas, muito obrigada pela sua avali...
response_text_normalized_sum     \n oie douglas, muito obrigada pela sua avali...
response_text_tokens_sum        [[], [], [], [], [], [], [], [], [], [], [], [...
highlights_sum                   \n Tranquilo, Ideal para crianças \n Vista li...
highlights_token

In [27]:
df_hotels.iloc[7]

name                                                          Boa Vista Eco Hotel
review_count                                                                 1139
other_language_count                                                           42
text_count                                                                    559
text_sum                        Deixa muito a desejar: serviço fraco, tv não f...
text_normalized_sum             deixa muito a desejar: servico fraco, tv nao f...
text_tokens_sum                 [[deixa, desejar, servico, fraco, nao, funcion...
response_text_count                                                             0
response_text_sum                                                             \n 
response_text_normalized_sum                                                  \n 
response_text_tokens_sum        [[], [], [], [], [], [], [], [], [], [], [], [...
highlights_sum                  Tranquilo \n Vista linda, Ambiente romântico, ...
highlights_token

# 4. Topic Modelling

In [28]:
save_df(df_prep, "data/prep", "df_text_prep")
save_df(df_hotels, "data/prep", "df_hotels")

data/prep/df_text_prep_2023-06-22_20-52-11_175568.pq
data/prep/df_hotels_2023-06-22_20-52-11_175568.pq


In [6]:
df_prep = pd.read_parquet("data/prep/df_text_prep_2023-06-22_20-52-11_175568.pq")
df_hotels = pd.read_parquet("data/prep/df_hotels_2023-06-22_20-52-11_175568.pq")

In [7]:
import numpy as np
text_tokens_flat = df_hotels.text_tokens_sum.apply(join_list)
text_tokens_flat.shape

(50,)

In [8]:
text_tokens_flat.head()

0    [parqu, ruim, desd, entrada, piscina, area, co...
1    [top, maravilhoso, quarto, maravilhoso, segura...
2    [quarto, pequena, bano, muy, incomodo, seguran...
3    [bom, preco, otima, localizacao, atendimento, ...
4    [fiquei, hospedada, gostei, estadia, lugar, tr...
Name: text_tokens_sum, dtype: object

In [9]:
from gensim import corpora

corpus = text_tokens_flat.to_list()
gensim_dictionary = corpora.Dictionary(corpus)

text_tokens_corpus = text_tokens_flat.progress_apply(lambda x:gensim_dictionary.doc2bow(x, allow_update=False))
gensim_corpus = text_tokens_corpus.to_list()

100%|██████████| 50/50 [00:00<00:00, 113.16it/s]


In [10]:
# Treinando modelo
import gensim
from gensim.models import CoherenceModel

lda_model = gensim.models.ldamodel.LdaModel(
    corpus=gensim_corpus, 
    num_topics=20, 
    id2word=gensim_dictionary, 
    passes=20,
    # per_word_topics=True
)

hdp_model = gensim.models.hdpmodel.HdpModel(
    corpus=gensim_corpus, 
    id2word=gensim_dictionary, 
    T=30
)

In [12]:
def perplexity_score(model: gensim.models.ldamodel.LdaModel):
    return model.log_perplexity(gensim_corpus)

print('LDA Perplexity:', perplexity_score(lda_model))

LDA Perplexity: -7.229908376737746


In [55]:
def coherence_score(model):
    coherence_score_model = CoherenceModel(model=model, texts=corpus, dictionary=gensim_dictionary, coherence='u_mass')
    return coherence_score_model.get_coherence()

print('LDA Coherence Score:', coherence_score(lda_model))
print('HDP Coherence Score:', coherence_score(hdp_model))

LDA Coherence Score: -0.05151109182430593
HDP Coherence Score: -11.597771208769716


In [ ]:
coherence_score_hdp = CoherenceModel(model=hdp_model, texts=corpus, dictionary=gensim_dictionary, coherence='c_v')
coherence_score = coherence_score_hdp.get_coherence()

print('HDP Coherence Score:', coherence_score(hdp_model))

In [16]:
# inspect the size of a python object in bytes
import joblib
joblib.dump(lda_model, "lda_model.joblib")
# 6mb

['lda_model.joblib']

In [25]:
def train_and_eval(n, alpha, beta):
    lda_model = gensim.models.ldamodel.LdaModel(
        corpus=gensim_corpus, 
        id2word=gensim_dictionary, 
        passes=10,
        chunksize=100,
        # per_word_topics=True,
        num_topics=n, 
        alpha=alpha,
        eta=beta,
    )
    p = perplexity_score(lda_model)
    c = coherence_score(lda_model)
    return {"n":n, "alpha":alpha, "beta":beta, "perplexity":p, "coherence":c}

In [32]:
import itertools
import pyperclip
import random
#list containing various hyperparameters
grid = {
    "s":[4,6,8,10,12,14,16,18,20],
    "alpha":['symmetric',0.3,0.5,0.7],
    "beta":['auto',0.3,0.5,0.7],
}


combinations_list = list(itertools.product(*grid.values()))
random.shuffle(combinations_list)
l = len(combinations_list)
print("len(combinations_list)", l)

# EARLY_STOP_CLIPBOARD_STRING

results = []
for i, params in tqdm(enumerate(combinations_list), total=l):
    results.append(train_and_eval(*params))
    print(results[-1])
    try:
        if pyperclip.paste() == "STOP_CLIPBOARD_STRING":
            print("Early stop via clipboard!")
            break
    except:
        pass

joblib.dump(results, "results/results.joblib")

len(combinations_list) 144


  1%|          | 1/144 [01:04<2:33:14, 64.30s/it]

{'n': 20, 'alpha': 'symmetric', 'beta': 0.5, 'perplexity': -7.150037288845718, 'coherence': 0.31945566271429476}
{'n': 20, 'alpha': 0.7, 'beta': 'auto', 'perplexity': -7.236617394342473, 'coherence': 0.31622345478719105}


  1%|▏         | 2/144 [02:21<2:50:25, 72.01s/it]

{'n': 14, 'alpha': 0.5, 'beta': 0.5, 'perplexity': -7.14748851886416, 'coherence': 0.32906760257808726}


...

{'n': 4, 'alpha': 0.7, 'beta': 0.5, 'perplexity': -7.138726313192562, 'coherence': 0.31156023973155333}


 99%|█████████▉| 143/144 [2:45:59<00:56, 56.76s/it]

{'n': 14, 'alpha': 0.5, 'beta': 'auto', 'perplexity': -7.102969693151883, 'coherence': 0.31430788320222}


100%|██████████| 144/144 [2:47:06<00:00, 69.63s/it]


['results/results.joblib']

In [34]:
df_gs = pd.DataFrame(results)
df_gs.sort_values("coherence", ascending=False).head(10)

,n,alpha,beta,perplexity,coherence
52,6,0.7,0.7,-7.163845,0.365298
85,10,symmetric,0.5,-7.150834,0.358531
73,8,0.7,0.7,-7.168303,0.353245
21,8,0.3,0.7,-7.161098,0.352712
98,10,0.7,0.7,-7.175720,0.337908
120,18,symmetric,0.5,-7.167740,0.334864
121,14,0.7,0.5,-7.145445,0.332878
43,10,symmetric,0.7,-7.171599,0.332289
122,10,0.5,0.7,-7.178286,0.331909
31,16,0.7,auto,-7.156709,0.331101


In [45]:
df_gs.sort_values("coherence", ascending=False).iloc[0]

n                    6
alpha              0.7
beta               0.7
perplexity   -7.163845
coherence     0.365298
Name: 52, dtype: object

In [47]:
# Treinando Modelo de melhor resultado
n = 6
alpha = 0.7
beta = 0.7
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=gensim_corpus, 
    id2word=gensim_dictionary, 
    passes=10,
    chunksize=100,
    # per_word_topics=True,
    num_topics=n, 
    alpha=alpha,
    eta=beta,
)

p = perplexity_score(lda_model)
c = coherence_score(lda_model)
print("Perplexity", p)
print("Coherence", c)

Perplexity -7.183829775690902
Coherence 0.3107981246556443


In [48]:
def print_topics_formatted(model, num_words=10):
    topics = lda_model.print_topics(num_words=num_words)
    for i, topic in enumerate(topics):
        t_str = topic[1]
        t_str = re.sub(r"[^a-zA-Z]", " ", t_str)
        t_str = re.sub(r"\s+", " ", t_str)
        t_str = f"Tópico {i}: {t_str}"
        print(t_str)

def print_topics(model, num_words=10):
    topics = hdp_model.print_topics(num_words=num_words)
    for topic in topics:
        print(topic)

In [53]:
# Imprimindo tópicos encontrados
print("LDA topics")
print_topics_formatted(lda_model, num_words=100)

LDA topics
Tópico 0:  nao bom hotel piscina lugar otimo quarto excelent atendimento bem sao boa pra dia tudo parqu maravilhoso funcionario comida todo otima familia recomendo melhor cafe agua manha muita local crianca pessoa super restaurant apartamento servico porem estrutura ate recepcao toda quent limpo limpeza poi gostei hora hosped pouco elevador aquatico preco check area top outro sempr cama ter pessimo desejar tambem opco ruim lindo experiencia resort grand estacionamento agradavel coisa nada ainda fila amei brinquedo educado perfeito falta lazer calda diversao almoco fica ambient fazer qualidad veze nova maravilhosa voce atencioso passar ponto varia organizado mal vale aconchegant apena fiquei 
Tópico 1:  atendimento excelent nao lugar bom comida todo equip otimo restaurant maravilhoso bem funcionario piscina boa tudo quarto resort sao hotel super ala atencioso melhor paraben dia recepcao toda otima praia maravilhosa nota lindo recomendo servico especi bebida sempr experiencia 

In [40]:
# Imprimindo tópicos encontrados
print("HDP topics")
print_topics_formatted(hdp_model)

HDP topics
Tópico 0:  parqu lugar bom otimo familia brinquedo nao diversao excelent dia 
Tópico 1:  hotel bom excelent atendimento nao lugar quarto otimo piscina bem 
Tópico 2:  excelent hotel atendimento quarto nao bom otimo todo restaurant lugar 
Tópico 3:  nao bom hotel lugar quarto excelent atendimento boa bem otimo 
Tópico 4:  bom nao lugar quarto atendimento excelent hotel comida otimo bem 
Tópico 5:  hotel cafe excelent atendimento manha quarto nao bom bem funcionario 
Tópico 6:  nao hotel bom piscina lugar otimo quarto excelent atendimento bem 
Tópico 7:  lugar excelent hotel bom atendimento nao maravilhoso otimo comida tudo 
Tópico 8:  hotel bom nao atendimento lugar quarto otimo excelent bem manha 
Tópico 9:  piscina lugar comida excelent bom atendimento funcionario otimo todo quarto 
Tópico 10:  nao quarto aeroporto hotel limpo banheiro bom hora bem sao 
Tópico 11:  hotel nao bom atendimento lugar otimo quarto excelent boa bem 
Tópico 12:  atendimento excelent nao lugar bom 

In [42]:
# Definindo função para atribuir topicos ao documento
def get_topics(tokens, model=lda_model):
    doc_bow = gensim_dictionary.doc2bow(tokens)
    doc_topics = model[doc_bow]
    doc_topics = sorted(doc_topics, key=lambda x:x[1], reverse=1)
    first_topic = doc_topics[0][0]
    return np.array(doc_topics), first_topic

get_topics(text_tokens_flat.iloc[2], model=lda_model)

(array([[13.        ,  0.88055509],
        [ 6.        ,  0.11726438]]),
 13)

In [43]:
df_prep[["lda_topics", "lda_first_topic"]] = pd.DataFrame(text_tokens_flat).progress_apply(
    lambda x: get_topics(x.text_tokens_sum, model=lda_model),
    axis=1,
    result_type="expand",
)

df_prep[["hdp_topics", "hdp_first_topic"]] = pd.DataFrame(text_tokens_flat).progress_apply(
    lambda x: get_topics(x.text_tokens_sum, model=hdp_model),
    axis=1,
    result_type="expand",
)

100%|██████████| 50/50 [00:01<00:00, 45.46it/s]


In [51]:
# Visualizando tópicos
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import matplotlib.pyplot as plt


In [52]:
print("LDA visualization")
lda_visualization = gensimvis.prepare(lda_model, gensim_corpus, gensim_dictionary)
pyLDAvis.display(lda_visualization)

LDA visualization


c:\Users\Bruno\miniconda3\envs\pln\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [47]:
print("HDP visualization")
hdp_visualization = gensimvis.prepare(hdp_model, gensim_corpus, gensim_dictionary)
pyLDAvis.display(hdp_visualization)

HDP visualization


c:\Users\Bruno\miniconda3\envs\pln\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


# Sentiment Analisys

In [322]:
from analysis.leia.leia import SentimentIntensityAnalyzer 

vader_model = SentimentIntensityAnalyzer()

# Análise de texto simples
vader_model.polarity_scores('Eu estou feliz')

{'neg': 0.0, 'neu': 0.328, 'pos': 0.672, 'compound': 0.6249}

In [327]:
df_prep.filter(regex="^text")

,text,text_is_other_language,text_tokens,text_tokens_len,text_normalized
0,"Bom preco ,Ótima Localização,atendimento bom ,...",False,"[[bom, preco, otima, localizacao, atendimento,...",4,"bom preco ,otima localizacao,atendimento bom ,..."
1,"Eu ,nunca vi um quarto de hotel tão sujo na mi...",False,"[[nunca, quarto, hotel, tao, sujo, vida], [len...",6,"eu ,nunca vi um quarto de hotel tao sujo na mi..."
2,"Hotel de baixíssima qualidade no atendimento, ...",False,"[[hotel, baixissima, qualidad, atendimento, ac...",3,"hotel de baixissima qualidade no atendimento, ..."
3,"Péssimo atendimento, muito clássico do Rio de ...",False,"[[pessimo, atendimento, classico, rio, janeiro...",3,"pessimo atendimento, muito classico do rio de ..."
4,,None,[[]],1,
...,...,...,...,...,...
270092,Recomendo a todos que desejam visitar gramado!...,False,"[[recomendo, todo, desejam, visitar, gramado, ...",1,recomendo a todos que desejam visitar gramado!...
270093,,None,[[]],1,
270094,muito gostoso e aconchegante. café da manhã ex...,False,"[[gostoso, aconchegant], [cafe, manha, excelen...",4,muito gostoso e aconchegante. cafe da manha ex...
270095,,None,[[]],1,


# Modelagem

Níveis:
* Hotel
* Avaliação
* Sentença

Sumarização Extrativa:
* N Melhores Sentenças

Modelagem de tópicos:
* Treino feito em cima das avaliações - Quais são os tópicos de uma avaliação?
* Predição feita em cima das sentenças

Análise de sentimento:
* Predição feita em cima das sentenças

Features da sentença:
* Tópicos da sentença
* Sentimento da sentença
* Usuário da avaliação é local guide
* Nota da avaliação
* Número de sentenças da avaliação
* Número de likes da avaliação


# Transformers

In [4]:
df_prep = pd.read_parquet("data/prep/df_text_prep_2023-06-22_20-52-11_175568.pq")
df_hotels = pd.read_parquet("data/prep/df_hotels_2023-06-22_20-52-11_175568.pq")

In [5]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bruno\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bruno\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# !python -m spacy download pt_core_news_lg

In [6]:
texto = df_hotels[df_hotels.name == "Copacabana Palace"].text_sum.str.replace(".", ";").str.replace("\n", ".").values[0]
texto

C:\Users\Bruno\AppData\Local\Temp\ipykernel_1640\2619961314.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  texto = df_hotels[df_hotels.name == "Copacabana Palace"].text_sum.str.replace(".", ";").str.replace("\n", ".").values[0]


'Maravilhosa Atividades próximas: Ótima Segurança: Muito bom Facilidade de deslocamento a pé: Sim Detalhes importantes: Muito bom mesmo . Sabe quando você sabe que está acontecendo algo realmente inesperado e de grande qualidade? Imagine que logo na entrada, logo na recepção você encontrará as melhores pessoas para te dar as boas vindas nesse maravilhoso palácio! Nessa tarde, em específico, estava acompanhada de um alemão e, para nossa surpresa, um dos recepcionistas que fica falava fluentemente alemão também! Foi maravilhoso! . Um restaurante perfeito de hotel de 100 anos em frente à praia de Copacabana ;  Tudo o que comemos estava delicioso ;  Não é barato, mas acho que o valor para essa equipe, ambiente e serviço de qualidade é baixo ;  Um dos chefs, Diego, foi incrivelmente amigável e positivo  ;  Todos ficaram muito satisfeitos com ele ;  Se quiserem se divertir, venham sem pensar ;  Já faz muito tempo que a refeição acabou, mas estamos saboreando nosso café à beira da piscina ;  

In [7]:
import spacy
from spacy.lang.pt.stop_words import STOP_WORDS
from collections import Counter
from tqdm import tqdm

# Carregar o modelo em português do spaCy
nlp = spacy.load('pt_core_news_lg')

# Tokenizar o texto em frases e palavras usando o spaCy
print("gerando doc")
doc = nlp(texto)
len(doc)

gerando doc


105370

In [39]:
print("gerando frases")
frases = [sent.text for sent in tqdm(doc.sents)]
print("gerando palavras")
palavras = [token.text for token in tqdm(doc) if not token.is_stop and token.is_alpha]

# Calcular a frequência de cada palavra no texto
frequencia_palavras = Counter(palavras)

# Calcular a pontuação de cada frase
pontuacao_frases = {}
print("calculando pontuação")
for frase in tqdm(frases,total=len(frases)):
    pontuacao = 0
    nlp_frase = nlp(frase)
    for palavra in nlp_frase:
        if palavra.text.lower() in frequencia_palavras:
            pontuacao += frequencia_palavras[palavra.text.lower()]
    pontuacao_frases[frase] = pontuacao

# Selecionar as frases mais importantes e gerar o resumo
num_frases_resumo = 2
frases_importantes = sorted(pontuacao_frases, key=pontuacao_frases.get, reverse=True)
resumo = "\n".join(frases_importantes[:num_frases_resumo])

# Imprimir o resumo
print("Resumo:")
print(resumo)

gerando frases


12246it [00:00, 119176.05it/s]


gerando palavras


100%|██████████| 105370/105370 [00:00<00:00, 1148394.32it/s]


calculando pontuação


100%|██████████| 12246/12246 [01:21<00:00, 149.73it/s]

Resumo:
O mais tradicional hotel do Rio de Janeiro ;  Hotel deslumbrante, funcionários muito cordiais, sempre se oferecendo para te atender, serviços impecáveis, quarto grande e mobiliário de muito bom gosto, piscina semi olímpica climatizada sensacional, ótima localização no bairro de Copacabana ;  Café da manhã muito bom com muitas variedades ;  Preço dos produtos oferecidos no hotel muito caros, mas condizente com o nível do hotel ;  
Excelente hotel, só tínhamos um mas já que o primeiro quarto que nos deram tinha comer cupim na área onde são colocadas as malas ;  a desvantagem ;  é um hotel que sempre voltamos apesar desse episódio, pois é o melhor hotel do Rio comparado aos outros ;  bom serviço em todos os sentidos, a equipe é super simpática  ;  Excelente vista da praia .


In [40]:
frases_importantes[:10]

['O mais tradicional hotel do Rio de Janeiro ;  Hotel deslumbrante, funcionários muito cordiais, sempre se oferecendo para te atender, serviços impecáveis, quarto grande e mobiliário de muito bom gosto, piscina semi olímpica climatizada sensacional, ótima localização no bairro de Copacabana ;  Café da manhã muito bom com muitas variedades ;  Preço dos produtos oferecidos no hotel muito caros, mas condizente com o nível do hotel ;  ',
 'Excelente hotel, só tínhamos um mas já que o primeiro quarto que nos deram tinha comer cupim na área onde são colocadas as malas ;  a desvantagem ;  é um hotel que sempre voltamos apesar desse episódio, pois é o melhor hotel do Rio comparado aos outros ;  bom serviço em todos os sentidos, a equipe é super simpática  ;  Excelente vista da praia .',
 'A conservação do hotel é boa, mas entendemos que alguns detalhes como móveis sem reforma, lustre e luminárias com problemas, luz na fechada do hotel piscando, atendimento cortês mas irregular falta de sinaliz

In [8]:
# Install spacy-transformers
# !pip install spacy-transformers

In [42]:
import spacy
from IPython.core.display import display, HTML

# Encontrando as entidades nomeadas e os substantivos mais importantes (que não são stopwords)
entidades_nomeadas = set([ent.text for ent in doc.ents if ent.label_ != ""])
substantivos = set([token.text for token in doc if token.pos_ == "NOUN" and token.is_stop == False])

# Destacando entidades nomeadas em vermelho e substantivos em azul
texto_formatado = texto
for entidade in entidades_nomeadas:
    texto_formatado = texto_formatado.replace(entidade, f"{entidade}")

for substantivo in substantivos:
    texto_formatado = texto_formatado.replace(substantivo, f"{substantivo}")

texto_formatado = "" + texto_formatado + ""
HTML(texto_formatado[:2000])

C:\Users\Bruno\AppData\Local\Temp\ipykernel_2168\491507696.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [48]:

# Calculando a pontuação de cada frase
frases_pontuadas = []
for i, frase in enumerate(tqdm(doc.sents)):
    num_entidades = len(set([ent.text for ent in frase.ents if ent.label_ != ""]).intersection(entidades_nomeadas))
    num_substantivos = len(set([token.text for token in frase if token.pos_ == "NOUN" and not token.is_stop]).intersection(substantivos))
    pontuacao = num_entidades + num_substantivos
    frases_pontuadas.append((frase, i, pontuacao))

# Ordenando as frases pela pontuação em ordem decrescente
frases_pontuadas = sorted(frases_pontuadas, key=lambda x: x[2], reverse=True)

# Definindo o número de frases a serem incluídas no resumo
num_frases_resumo = 3

# Selecionando as frases mais importantes para o resumo
frases_importantes = [frase for frase, _, _ in frases_pontuadas[:num_frases_resumo]]

12246it [01:42, 119.49it/s]


In [49]:
frases_importantes

[Fomos tomar café da manhã no Copacabana Palace e foi uma experiência extremamente satisfatória em TODOS os sentidos,Lugar muito aconchegante ficamos a princípio dentro do Restaurante que e muito Bonito e como queria viver toda a experiência pedi gentilmente para trocar de lugar e nos sentarmos também na mesa ao lado da piscina o dia estava nublado e tempo agradável e Presizo ressaltar que o Garson Francisco foi extremamente Solicito e ainda nos ajudou a levar copos e pratinho para essa outra mesa com muita simpatia  ;  Estamos em três e minha filha ainda tinham sonhos que por sinal eu também comi e são os melhores que já degustei em toda minha Vida todas os outros produtos são EXCELENTES coxinhas outros salgados Folheados e Fritos mas tudo bem sequinho e com apresentação IMPECÁVEL Pães de TODOS OS TIPOS QUE EXISTE EU ACHO TUDO EXATAMENTE TUDO FOI MUITO BOM agora vem a parte dolorosa e Aniversário da Mulher e Ela merece mesmo e se hospedar nesse lugar muito Incrível Valor que vale a pe

In [13]:
from transformers import pipeline

summarizer = pipeline("summarization", model="phpaiola/ptt5-base-summ-wikilingua")


In [ ]:
texto

In [15]:
len(texto)

510729

In [24]:
df_hotels

,name,review_count,other_language_count,text_count,text_sum,text_normalized_sum,text_tokens_sum,response_text_count,response_text_sum,response_text_normalized_sum,response_text_tokens_sum,highlights_sum,highlights_tokens_sum
0,Acqua Lokos,8542,350,3994,"O parque tá muito ruim,desde a entrada!Piscina...","o parque ta muito ruim,desde a entrada!piscina...","[[parqu, ruim, desd, entrada, piscina, area, c...",126,"\n Oie Douglas, muito obrigada pela sua avali...","\n oie douglas, muito obrigada pela sua avali...","[[], [], [], [], [], [], [], [], [], [], [], [...","\n Tranquilo, Ideal para crianças \n Vista li...","[[], [], [], [], [], [], [tranquilo, ideal, cr..."
1,Aroso Paço Hotel,1118,22,660,"Top, maravilhoso Quartos: Maravilhoso Seguranç...","top, maravilhoso quartos: maravilhoso seguranc...","[[top, maravilhoso, quarto, maravilhoso, segur...",4,"\n Prezado Senhor Fernando, Agradecemos a ava...","\n prezado senhor fernando, agradecemos a ava...","[[], [], [], [], [], [], [], [], [], [], [], [...","Luxuoso, Vista linda, Ambiente romântico, Tran...","[[luxuoso, vista, linda, ambient, romantico, t..."
2,Atlantic Hotel Copacabana,4983,748,2857,\n Quartos: Pequeña . baño muy incómodo Segu...,\n quartos: pequena . bano muy incomodo segu...,"[[], [quarto, pequena], [bano, muy, incomodo, ...",0,\n,\n,"[[], [], [], [], [], [], [], [], [], [], [], [...","Vista linda, Bom preço \n Luxuoso, Tranquilo, ...","[[vista, linda, bom, preco], [], [], [], [], [..."
3,Atlântico Center,1397,84,857,"Bom preco ,Ótima Localização,atendimento bom ,...","bom preco ,otima localizacao,atendimento bom ,...","[[bom, preco, otima, localizacao, atendimento,...",0,\n,\n,"[[], [], [], [], [], [], [], [], [], [], [], [...","Bom preço \n Bom preço \n Tranquilo, Bom preço...","[[bom, preco], [], [], [bom, preco], [], [tran..."
4,Atlântico Inn Apart Hotel,1471,35,849,"Fiquei hospedada, gostei da estadia, lugar tra...","fiquei hospedada, gostei da estadia, lugar tra...","[[fiquei, hospedada, gostei, estadia, lugar, t...",0,\n,\n,"[[], [], [], [], [], [], [], [], [], [], [], [...","Tranquilo, Bom preço \n Tranquilo, Bom preço \...","[[tranquilo, bom, preco], [], [], [], [], [tra..."
5,Atrium Quinta de Pedras,1392,43,802,"\n Atendeu plenamente nossas necessidades, co...","\n atendeu plenamente nossas necessidades, co...","[[], [atendeu, plenament, necessidad, quarto, ...",0,\n,\n,"[[], [], [], [], [], [], [], [], [], [], [], [...","\n Tranquilo \n Luxuoso, Ambiente romântico, ...","[[], [tranquilo], [luxuoso, ambient, romantico..."
6,Blue Tree Thermas de Lins,5032,74,3146,"Ótima \n Acomodação, refeição, atendimento são...","otima \n acomodacao, refeicao, atendimento sao...","[[otima], [acomodacao, refeicao, atendimento, ...",1815,"\n Prezado Henrique, Nós do Blue Tree Thermas...","\n prezado henrique, nos do blue tree thermas...","[[], [], [], [], [], [prezado, henriqu, blue, ...","\n Vista linda, Tranquilo, Ideal para criança...","[[], [vista, linda, tranquilo, ideal, crianca]..."
7,Boa Vista Eco Hotel,1139,42,559,"Deixa muito a desejar: serviço fraco, tv não f...","deixa muito a desejar: servico fraco, tv nao f...","[[deixa, desejar, servico, fraco, nao, funcion...",0,\n,\n,"[[], [], [], [], [], [], [], [], [], [], [], [...","Tranquilo \n Vista linda, Ambiente romântico, ...","[[tranquilo], [], [], [], [], [vista, linda, a..."
8,Bourbon Cataratas do Iguaçu Thermas Eco Resort,4868,495,2471,"Resort de ótima qualidade, funcionários bem at...","resort de otima qualidade, funcionarios bem at...","[[resort, otima, qualidad, funcionario, bem, a...",562,"\n Prezado Nuno, Obrigada por dedicar um temp...","\n prezado nuno, obrigada por dedicar um temp...","[[], [], [], [], [prezado, nuno, obrigada, ded...","\n Luxuoso \n Tranquilo, Ideal para crianças ...","[[], [luxuoso], [tranquilo, ideal, crianca], [..."
9,Caravelle Palace Hotel,3208,60,1744,"\n Quarto limpo e ajeitado, café da manhã bom...","\n quarto limpo e ajeitado, cafe da manha bom...","[[], [], [quarto, limpo, ajei

hotel
avaliação 
frase - na análise de sentimento pode fazer sentido quebrar por frase


passar análise em todos os textos para avaliar a quantidade de neutros

In [18]:
summary = summarizer(texto[:10_000])
print(summary)

[{'summary_text': 'Aproveite o hotel! Aproveite o hotel! Aproveite o'}]


In [22]:
df_hotels.text_count.sum() / 50

3015.48

In [19]:
df_hotels.name

0                                          Acqua Lokos
1                                     Aroso Paço Hotel
2                            Atlantic Hotel Copacabana
3                                     Atlântico Center
4                            Atlântico Inn Apart Hotel
5                              Atrium Quinta de Pedras
6                            Blue Tree Thermas de Lins
7                                  Boa Vista Eco Hotel
8       Bourbon Cataratas do Iguaçu Thermas Eco Resort
9                               Caravelle Palace Hotel
10                                 Castro's Park Hotel
11                                   Copacabana Palace
12                       Costa do Sauípe - Premium Sol
13                           Costão do Santinho Resort
14                                    Céu Palmas Hotel
15                Dayrell Hotel & Centro de Convenções
16                                          Fast Sleep
17    Golden Dolphin Grand Hotel - Central de Reservas
18        

verificar se tem pelo menos um substantivo e um adjetivo

fazer fluxograma do tratamento de texto
Mostrar filtrando os textos

lda: colocar lista de keywords no chat gpt para ele dar o label

Bom ponto de partida
